In [1]:
# print version / should be 1.6.3
println(versioninfo())

# load centralized paths dictionary
using YAML
PATHS_FILE = "../../../PATHS.yml"
PATHS = YAML.load_file(PATHS_FILE);

"""
    load_path(keys::Vector{String}):String

Return the absolute path for a requested item within the nested PATHS dictionary.
"""
function loadpath(keys::Vector{String}):String
    # recusively assemble paths from keys
    requested_path = foldl((x, y) -> getindex(x, y), keys; init=PATHS)
    # rephrase abs path from relative paths to be platform independent.
    realpath(joinpath(splitdir(realpath(PATHS_FILE))[1], requested_path))
end;

Julia Version 1.6.3
Commit ae8452a9e0 (2021-09-23 17:34 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2630 v3 @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  __LMOD_REF_COUNT_JULIA_DEPOT_PATH = /home/biotoml/.julia:1;/sw/comp/julia/1.6.3/rackham/lib/glob_pkg:1
  JULIA_DEPOT_PATH = /home/biotoml/.julia:/sw/comp/julia/1.6.3/rackham/lib/glob_pkg
  __LMOD_REF_COUNT_JULIA_LOAD_PATH = @:1;@v#.#:1;@stdlib:1;/sw/comp/julia/1.6.3/rackham/lib/glob_pkg/environments/v1.6:1
  JULIA_PROJECT = /crex/proj/snic2022-23-321/private/thomas/Isoprene/ext_src/01_julia_land_v01/Land/Project.toml
  JULIA_ROOT = /sw/comp/julia/1.6.3/rackham
  JULIA_LOAD_PATH = @:@v#.#:@stdlib:/sw/comp/julia/1.6.3/rackham/lib/glob_pkg/environments/v1.6
nothing


In [2]:
# we load the project path as it is used by CliMA Land v01 papers
using Pkg
PROJECT_PATH = loadpath(["EXTERNAL", "01_CLIMA_LAND_V01", "ROOT"])
Pkg.activate(PROJECT_PATH)

  Activating environment at `/crex/proj/snic2022-23-321/private/thomas/Isoprene/ext_src/01_julia_land_v01/Land/Project.toml`


In [2]:
using LazyArtifacts

using CanopyLayers: BLUE, EVI, FourBandsFittingHybrid, NDVI, NIR, NIRv, NIRvES, RED, SIF_740, SIF_WL, SIF_fluxes!, canopy_fluxes!, canopy_geometry!, canopy_matrices!, fit_soil_mat!, short_wave!, soil_albedos
using DataFrames: DataFrame
using GriddingMachine: lat_ind, lon_ind
using Photosynthesis: AbstractPhotoModelParaSet, AirLayer, leaf_ETR!
using PkgUtility: numerical∫, read_csv, save_csv!
using PlantHydraulics: soil_p_25_swc, temperature_effects!
using SoilPlantAirContinuum: SPACMono, initialize_spac_canopy!, update_Cab!, update_LAI!, zenith_angle
using Statistics: mean
using StomataModels: BetaGLinearPsoil, CanopyLayer, ESMMedlyn, EmpiricalStomatalModel, GswDrive, gas_exchange!, gsw_control!, stomatal_conductance, update_leaf_TP!, β_factor
using UnPack: @unpack
using WaterPhysics: saturation_vapor_pressure

In [5]:
using PlantHydraulics: create_grass

In [6]:
methods(create_grass)

# 1 method for generic function "create_grass":
[1] create_grass(z_root::FT, z_canopy::FT, soil_bounds::Vector{FT}, air_bounds::Vector{FT}) where FT<:AbstractFloat in PlantHydraulics at /home/biotoml/.julia/packages/PlantHydraulics/Aztny/src/initialize/plant.jl:20

In [ ]:
# function to create a SPAC node
function create_spac(
        lat::FT,  # core
        lon::FT,  # core
        # elevation::FT,  # can be computed from lat+lon
        
        
        _soil_bnds::AbstractVector{FT} = FT[0, -0.1, -0.35, -1, -3],  # 4 layers of soil with given boundaries (in m) as for JULES - can use ERA5 data
        _z_c::FT = 5.0,  # canopy height should be used from langHighresolutionCanopyHeight2022 or GriddingMachine
        _z_r::FT = -2,  # we set a default 2 m max root depth. Closest reference is fanHydrologicRegulationPlant2017 and https://wci.earth2observe.eu/thredds/catalog/usc/root-depth/catalog.html
        n_airlayers::Int8 = 20,  # not more than 127!
        
        # following components can be taken from GriddingMachine / ERA5
        wind_z0::FT,
        wind_d::FT,
        wind_zs::Vector{FT},
        winds::Vector{FT},
        mswc::Vector{FT},
        swc::Vector{FT},
        p_soil::Vector{FT},
        h_soil::FT
        lai::FT,  # leaf area index from 
        
        # photomodel C3 or C4
        #  can be C3VJPModel, C3CytochromeModel, C4VJPModel
        plant_hs::AbstractPlantOrganism{FT} = create_grass(...)  # vegetation_type - GrassLikeOrganism, TreeLikeOrganism, PalmLikeOrganism (init via create_*)
        
        # Stomatal models - we can try different models - optimization approaches were great in trials
        stomata_model::AbstractStomatalModel{FT} = ESMMedlyn{FT}(),
        
        
        ) where {FT<:AbstractFloat}
    _Δz::FT    = _z_c / n_airlayers;
    _air_bnds  = collect(FT, 0:_Δz:_z_c+2*_Δz);
    _elev::FT  = 100;


    # create a SPACMono struct
    _node = SPACMono{FT}(soil_bounds = _soil_bnds, air_bounds = _air_bnds, z_canopy = _z_c, z_root = _z_r, latitude = lat, longitude = lon, elevation = _elev, stomata_model = ESMMedlyn{FT}());

    # initialize the canopy RT model
    initialize_spac_canopy!(_node);

    return _node

end;




In [6]:
typemax(Int16)

32767